In [1]:
from pymongo import MongoClient
import pandas as pd

In [19]:
conn = MongoClient()
db = conn.Zillow
data = db.housing_description
df = pd.DataFrame(list(data.find()))
conn.close()

In [107]:
from datetime import datetime
def string_to_date(s):
    try:
        return datetime.strptime(s, '%m/%d/%y')
    except:
        return None
def get_listed_for_sale(price_history):
    try:
        if type(price_history)!=list:
            return None
        for change in price_history:
            if change[1] == 'Listed for sale':
                return string_to_date(change[0].replace('\\',''))
    except:
        pass
    return None

In [85]:
#drop all rows that do not have a price history
df = df.dropna(subset=['price_history'])
df = df.reset_index(drop=True)

In [109]:
df['listed_for_sale'] = df['price_history'].apply(get_listed_for_sale)
df['sold_on'] = df['sold_on'].apply(string_to_date)

In [110]:
df['days_on_market'] = df['sold_on'] - df['listed_for_sale']

In [116]:
df = df[df.days_on_market.notnull()]

In [ ]:
df.to_pickle('current.p')